In [8]:
import hail as hl
import numpy as np
from hail.methods import export_plink
from cpg_utils import to_path
from pandas_plink import read_plink1_bin

In [9]:
from cpg_utils.hail_batch import dataset_path, init_batch

SyntaxError: invalid syntax (hail_batch.py, line 150)

In [6]:
MT = dataset_path('v0/VPREB3_50K_window.mt')

NameError: name 'dataset_path' is not defined

In [7]:
init_batch()
# read hail matrix table object (WGS data)
mt = hl.read_matrix_table(MT)

NameError: name 'init_batch' is not defined

In [ ]:
# densify
mt = hl.experimental.densify(mt)

In [ ]:
# filter out low quality variants and consider biallelic variants only (no multi-allelic, no ref-only)
mt = mt.filter_rows(
        (hl.len(hl.or_else(mt.filters, hl.empty_set(hl.tstr))) == 0)
        & (hl.len(mt.alleles) == 2)
        & (hl.is_snp(mt.alleles[0], mt.alleles[1]))
    )

In [ ]:
# filter rare variants only (freq < 5%)
mt = hl.variant_qc(mt)
mt = mt.filter_rows((mt.variant_qc.AF[1] < 0.05) & (mt.variant_qc.AF[1] > 0))

In [ ]:
# select further (only 10 variants to test)
mt = mt.head(10)
# export MT object to PLINK (all regulatory variants)
mt_path = output_path('SKAT/v0/test')
export_plink(mt, mt_path, ind_id=mt.s)

# read in genotype file (plink format)
genotype_file_bed = "gs://cpg-tob-wgs-test/SKAT/v0/test.bed"
genotype_file_bim = "gs://cpg-tob-wgs-test/SKAT/v0/test.bim"
genotype_file_fam = "gs://cpg-tob-wgs-test/SKAT/v0/test.fam"
# bed
with to_path(genotype_file_bed).open('rb') as handle:
    data = handle.readlines()
with open('temp.bed', 'wb') as handle:
    handle.writelines(data)
# bim
with to_path(genotype_file_bim).open('rb') as handle:
    data = handle.readlines()
with open('temp.bim', 'wb') as handle:
    handle.writelines(data)
# fam
with to_path(genotype_file_fam).open('rb') as handle:
    data = handle.readlines()
with open('temp.fam', 'wb') as handle:
    handle.writelines(data)
# read
G = read_plink1_bin('temp.bed')

np.nanmin(G.values)
np.nanmean(G.values)
np.nanmax(G.values)